In [1]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/smainewdataset/Phase_2_data/labels_train.csv")
lat_mean, lat_std = train_df.latitude.mean(), train_df.latitude.std()
lon_mean, lon_std = train_df.longitude.mean(), train_df.longitude.std()

print(f"LAT_MEAN = {lat_mean:.6f}")
print(f"LAT_STD  = {lat_std:.6f}")
print(f"LON_MEAN = {lon_mean:.6f}")
print(f"LON_STD  = {lon_std:.6f}")


LAT_MEAN = 227431.775145
LAT_STD  = 753172.258700
LON_MEAN = 142366.444207
LON_STD  = 14900.627910


In [3]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# === Paths & Hyperparams ===
TRAIN_CSV    = "/kaggle/input/smainewdataset/Phase_2_data/labels_train.csv"
VAL_CSV      = "/kaggle/input/smainewdataset/Phase_2_data/labels_val.csv"
VAL_IMG_DIR  = "/kaggle/input/smainewdataset/Phase_2_data/images_val/images_val"
MODELS_DIR   = "/kaggle/input/regionmodels/tensorflow2/default/1"
OUTPUT_CSV   = "val_predictions.csv"
BATCH_SIZE   = 32
NUM_WORKERS  = 4

# === 1) Train‐set stats for standardization and cyclic bounds ===
train_df = pd.read_csv(TRAIN_CSV)
lat_mean, lat_std = train_df.latitude.mean(), train_df.latitude.std()
lon_mean, lon_std = train_df.longitude.mean(), train_df.longitude.std()
train_min_lat, train_max_lat = train_df.latitude.min(), train_df.latitude.max()
train_min_lon, train_max_lon = train_df.longitude.min(), train_df.longitude.max()

def denstd_lat(x): return x * lat_std + lat_mean
def denstd_lon(x): return x * lon_std + lon_mean

# === 2) Load & augment validation DataFrame ===
val_df = pd.read_csv(VAL_CSV)

def add_cyclic(df):
    df['lat_sin'] = np.sin(2*np.pi * (df.latitude - train_min_lat) / (train_max_lat - train_min_lat))
    df['lat_cos'] = np.cos(2*np.pi * (df.latitude - train_min_lat) / (train_max_lat - train_min_lat))
    df['lon_sin'] = np.sin(2*np.pi * (df.longitude - train_min_lon) / (train_max_lon - train_min_lon))
    df['lon_cos'] = np.cos(2*np.pi * (df.longitude - train_min_lon) / (train_max_lon - train_min_lon))
    return df

val_df = add_cyclic(val_df)

# === 3) Dataset ===
META = ['lat_sin','lat_cos','lon_sin','lon_cos']

class GeoValDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.tf = transform or transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],
                                 [0.229,0.224,0.225]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Image
        img = Image.open(os.path.join(self.img_dir, row.filename)).convert("RGB")
        img = self.tf(img)

        # Meta: ensure float32 numpy before tensor
        meta_vals = row[META].astype(np.float32).to_numpy()
        meta = torch.from_numpy(meta_vals)

        # True standardized lat/lon
        lat_t = torch.tensor((row.latitude - lat_mean) / lat_std, dtype=torch.float32)
        lon_t = torch.tensor((row.longitude - lon_mean) / lon_std, dtype=torch.float32)

        return img, meta, lat_t, lon_t, int(row.Region_ID), row.filename

# === 4) FusionModel ===
class FusionModel(nn.Module):
    def __init__(self, backbone, meta_dim=4, embed_dim=768):
        super().__init__()
        self.backbone = backbone
        self.meta_net = nn.Sequential(
            nn.Linear(meta_dim, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        self.head = nn.Sequential(
            nn.Linear(embed_dim + 64, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2),
        )

    def forward(self, x, m):
        feats = self.backbone(x)
        cls_tok = feats[:, 0]
        mfeat   = self.meta_net(m)
        return self.head(torch.cat([cls_tok, mfeat], dim=1))

# === 5) Load backbone & all region‐specific FusionModels ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# DINOv2 backbone
dino = torch.hub.load('facebookresearch/dinov2:main', 'dinov2_vitb14', pretrained=True)
dino.eval()
backbone = nn.Sequential(dino.patch_embed, *dino.blocks, dino.norm).to(device)

# Load each best_region_{r}.pth
fmodels = {}
for fname in os.listdir(MODELS_DIR):
    if fname.startswith("best_region_") and fname.endswith(".pth"):
        r = int(fname.split("_")[-1].split(".")[0])
        fm = FusionModel(backbone).to(device)
        fm.load_state_dict(torch.load(os.path.join(MODELS_DIR, fname), map_location=device))
        fm.eval()
        fmodels[r] = fm

# === 6) Inference, metrics, and CSV output ===
def main():
    ds     = GeoValDataset(val_df, VAL_IMG_DIR)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    records = []
    with torch.no_grad():
        for imgs, metas, lat_t, lon_t, region_true, fnames in loader:
            imgs, metas = imgs.to(device), metas.to(device)

            for i in range(imgs.size(0)):
                r = int(region_true[i])
                fm = fmodels[r]
                out = fm(imgs[i:i+1], metas[i:i+1])[0].cpu().numpy()

                records.append({
                    "filename"   : fnames[i],
                    "region_true": r,
                    "lat_true"   : float(lat_t[i].item() * lat_std + lat_mean),
                    "lon_true"   : float(lon_t[i].item() * lon_std + lon_mean),
                    "lat_pred"   : float(denstd_lat(out[0])),
                    "lon_pred"   : float(denstd_lon(out[1])),
                })

    # Convert to DataFrame
    df = pd.DataFrame(records)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved {len(df)} rows to {OUTPUT_CSV}")

    # Compute MSEs
    mse_lat = ((df.lat_true - df.lat_pred) ** 2).mean()
    mse_lon = ((df.lon_true - df.lon_pred) ** 2).mean()
    avg_mse = 0.5 * (mse_lat + mse_lon)

    # Print metrics
    print(f"MSE Latitude : {mse_lat:.6f}")
    print(f"MSE Longitude: {mse_lon:.6f}")
    print(f"Average MSE  : {avg_mse:.6f}")

if __name__ == "__main__":
    main()


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
/tmp/ipykernel_215/1714561270.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fm.load_state_di

Saved 369 rows to val_predictions.csv
MSE Latitude : 694191563456.428101
MSE Longitude: 386358324.799213
Average MSE  : 347288960890.613647
